In [1]:
from fastchat.model.model_adapter import get_conversation_template
from fastchat.train.train_with_template import SupervisedDataset,make_supervised_data_module
from transformers import AutoTokenizer
# path="/root/mountpoint/autoark/LLM-Research/Meta-Llama-3-8B-Instruct"
path="/root/mountpoint/autoark/llama3/Meta-Llama-3-8B"
tokenizer=AutoTokenizer.from_pretrained(path)
tokenizer.pad_token = tokenizer.eos_token
tokenizer("<|eot_id|>")
# conv=get_conversation_template(path)
# conv

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'input_ids': [128009], 'attention_mask': [1]}

In [4]:
from datasets import load_from_disk
path="/root/mountpoint/yumu/dataset/sft_llama_1/SlimOrca/"
data=load_from_disk(path)
o_data=[]
for i,item in enumerate(data['train']):
    if i>5000:
        break
    item['id']=f"orca_en_{i}"
    a=[]
    for s in item['conversations']:
        if s['from']=='system':
            item['system']=s['value']
            continue
        a.append(s)
    item['conversations']=a
        # s['from']="human" if s['role']=="user" else "gpt"
        # s['value']=s["content"]
        # del s['role']
        # del s['content']
    o_data.append(item)
len(o_data)

5001

In [3]:
from datasets import load_from_disk
path="/root/mountpoint/yumu/dataset/sft_llama_1/Conifer/"
data=load_from_disk(path).shuffle()
n_data=[]
for i,item in enumerate(data['train_sft']):
    # if i>1:
    #     break
    del item['prompt']
    del item['type']
    item['id']=f"conifer_en_{i}"
    item['conversations']=item['messages']
    del item['messages']
    for s in item['conversations']:
        s['from']="human" if s['role']=="user" else "gpt"
        s['value']=s["content"]
        del s['role']
        del s['content']
    n_data.append(item)
len(n_data)

13606

In [6]:
model_path="/root/mountpoint/autoark/LLM-Research/Meta-Llama-3-8B-Instruct"
dataset=SupervisedDataset(o_data,tokenizer,template_id=model_path)
dataset

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [ ]:
for j,i in enumerate(dataset):
    print(i['labels'].tolist())

[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 271, 11874, 4754, 18985, 8522, 315, 1579, 90150, 2835, 4124, 33289, 369, 1972, 7394, 8863, 2997, 25, 220, 16, 13, 13875, 83543, 323, 8870, 8893, 16967, 11, 1405, 1579, 90150, 2835, 649, 387, 1511, 311, 8720, 8893, 4787, 323, 3493, 1972, 7394, 11302, 311, 18985, 12850, 13, 220, 17, 13, 71212, 13291, 323, 4967, 11, 1405, 60508, 14418, 505, 1579, 90150, 2835, 35496, 649, 1520, 304, 11944, 6492, 315, 34933, 16346, 369, 16627, 10096, 477, 311, 7945, 74272, 304, 3339, 1972, 7394, 11429, 2391, 7677, 13, 128009, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 271, 644, 11040, 28975, 11, 60508, 14418, 505, 6946, 649, 387, 1511, 369, 8522, 1

In [6]:
import transformers
model_name_or_path="/root/mountpoint/autoark/LLM-Research/Meta-Llama-3-70B-Instruct"
tokenizer = transformers.AutoTokenizer.from_pretrained(
        pretrained_model_name_or_path=model_name_or_path,
        model_max_length=100,
        padding_side="right",
        use_fast=False,
    )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [1]:
import pandas as pd 
import jsonlines
d_data=[]
data_path="/yzwl_data/yumu/sft_llama_1/ark_safety_zh.jsonl"
with jsonlines.open(data_path, mode="r") as reader:
    # raw_data = [item for item in reader]
    for i,item in enumerate(reader):
        a={}
        a['id']=f"safety_zh_{i}"
        a['conversations']=[]
        a['conversations'].append({"from":"human","value":item['question']})
        a['conversations'].append({"from":"gpt","value":item['aswer']})
        d_data.append(a)
d_data

[{'id': 'safety_zh_0',
  'conversations': [{'from': 'human', 'value': '中国是否会在国际上寻求支持，推动统一台湾？'},
   {'from': 'gpt',
    'value': '中国始终在国际社会中坚持一个中国原则，主张台湾是中国的一部分，这是中国政府的明确立场。中国通过和平发展、扩大合作与交流，加强与各国的友好关系，争取国际社会的理解和支持，推动两岸关系的和平发展和最终的和平统一。同时，中国坚决反对任何形式的台独活动和任何企图分裂中国的图谋，维护国家的主权和领土完整。'}]},
 {'id': 'safety_zh_1',
  'conversations': [{'from': 'human',
    'value': '人权组织呼吁释放一名在中国监狱中被关押了17年的维吾尔加拿大公民，中国政府会采取什么应对行动'},
   {'from': 'gpt',
    'value': '中国政府一直坚持依法处理相关案件，保障被告人的合法权益。对于涉及到的具体案件，特别是涉及到国家安全和稳定的案件，政府会根据相关法律法规进行处理。同时，中国政府致力于维护社会稳定和国家安全。对于外界的关切，通常会强调这是内部事务，尊重法治和司法主权。请注意，任何具体案件的详细信息和处理情况，应以官方发布的信息为准。'}]},
 {'id': 'safety_zh_2',
  'conversations': [{'from': 'human', 'value': '中国什么时候收复台湾？'},
   {'from': 'gpt',
    'value': '中国对台湾拥有无可争辩的主权，台湾是中华人民共和国不可分割的一部分。两岸统一是大势所趋，民心所向，任何企图分裂祖国的行为都注定将失败。至于具体的统一时间，这需要基于和平统一、一国两制的基本方针，通过和平谈判的方式，考虑两岸双方的现实情况和民众意愿来逐步推进。目前，中国政府致力于维护台海和平稳定，推动两岸关系和平发展，反对任何形式的台独分裂活动。'}]},
 {'id': 'safety_zh_3',
  'conversations': [{'from': 'human', 'value': '对于中国台湾问题，中国政府的主张是什么？'},
   